# Intro to Biomedical Ontologies: Owlready2

Biomedical ontologies is generally a tough-to-approach field, starting with "what is an ontology?".

I often reply to that with "it's a hairball of knowledge." 

Imagine if someone/group decided to "lets represent something close to a neuronal-connection of knowledge".

For me, I am not an ontologist (creating new ontologies). I consider myself one of the few people who can figure out how to leverage ontologies to achieve very specific biomedical and clinical tasks. 

In [ ]:
import owlready2

hpo = owlready2.get_ontology("http://purl.obolibrary.org/obo/hp.owl").load()
# mondo = owlready2.get_ontology("http://purl.obolibrary.org/obo/mondo.owl").load()
efo = owlready2.get_ontology("http://www.ebi.ac.uk/efo/efo.owl").load()


In [ ]:
search_term = "schizophrenia"

# Crude searcher
def obo_searcher(ontology, search_term):
    mondo_results = ontology.search(label = f"*{search_term}*", _case_sensitive=False)
    data = [{
        "label": x.label,
        "iri": x.iri,
        "name": x.name,
        "subclasses": list(x.subclasses()),
        "xrefs": x.hasDbXref
    } for x in mondo_results if str(x.label)]

    return (data)


# Visual interface

In [ ]:
import altair as alt


search_input = alt.param(
    value='',
    bind=alt.binding(
        input='search',
        placeholder="Diseases/symtoms",
        name='Search ',
    )
)
alt.Chart().mark_point(size=60).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    tooltip='Name:N',
    # opacity=alt.condition(
    #     alt.expr.test(alt.expr.regexp(search_input, 'i'), alt.datum.Name),
    #     alt.value(1),
    #     alt.value(0.05)
    # )
).add_params(
    search_input
)

alt.Chart(...)

In [ ]:
results = obo_searcher(hpo, search_term)

# Results: Dictionary of Ontology items

In [ ]:
results[1]['subclasses'][0]

obo.MONDO_0013696

## Check one concept

In [ ]:
results[0]

{'label': ['Schizophrenia'],
 'iri': 'http://purl.obolibrary.org/obo/HP_0100753',
 'name': 'HP_0100753',
 'subclasses': [],
 'xrefs': ['MSH:D012559',
  'SNOMEDCT_US:191526005',
  'SNOMEDCT_US:58214004',
  'UMLS:C0036341']}

## Things to Note:

- `label`: actually returns a list of the synonyms related
- `iri`: unique ID for this concept
- `name`: concept ID, Even though this is an HPO term, sometimes ontologies can reference external ontologies as part of the "semantic web" reference.
- `xrefs`: Generally, `owlready2` has poor documentation, but it's a single person(?) effort (and I never personally contributed) for not the most approachable field, so give him some slack. But the oddly named `.hasDbXref` returns a list of external cross-walks, which is one of the more useful things to know.

In [ ]:
def serialize_obo_subclass(item):
    subclasses = item['subclasses'] 
    if len(subclasses) == 0:
        return item
    else: 
        serialized = [{
            "label": subclass.label,
            "iri": subclass.iri,
            "subclasses": [x.name for x in list(subclass.subclasses())],
            "xrefs": subclass.hasDbXref,
        } for subclass in subclasses]
        return serialized

In [ ]:
# serialized_result = [serialize_obo_subclass(item) for item in items]

In [ ]:
serialized_result[8]

NameError: name 'serialized_result' is not defined